# Deploying Machine Learning Models using kubectl on Minikube (no RBAC)
This demo shows how you can interact directly with kubernetes using kubectl to create and manage runtime machine learning models. It uses Minikube as the target Kubernetes cluster.
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [Helm](https://github.com/kubernetes/helm)
 - [Minikube](https://github.com/kubernetes/minikube) version v0.24.0 or greater
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)

Start minikube and ensure custom resource validation is activated and ther is 5G of memory.

In [6]:
!minikube start --memory=5000 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=AlwaysAllow

Starting local Kubernetes v1.9.4 cluster...
Starting VM...
Getting VM IP address...
Moving files into cluster...
Setting up certs...
Connecting to cluster...
Setting up kubeconfig...
Starting cluster components...
Kubectl is now configured to use the cluster.
Loading cached images from config file.


Install Helm

In [7]:
!helm init

$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.
Happy Helming!


Label the node to allow load testing to run on it

In [8]:
!kubectl label nodes `kubectl get nodes -o jsonpath='{.items[0].metadata.name}'` role=locust --overwrite

node "minikube" labeled


## Start seldon-core

Install the custom resource definition

In [10]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd \
    --set usage_metrics.enabled=true \
    --set rbac.enabled=false

NAME:   seldon-core-crd
LAST DEPLOYED: Mon Jun 11 19:56:24 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     0s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        1        1           0          0s

==> v1/Pod(related)
NAME                                         READY  STATUS             RESTARTS  AGE
seldon-spartakus-volunteer-5977c9fdb8-kr74m  0/1    ContainerCreating  0         0s


NOTES:
NOTES: TODO




In [11]:
!kubectl create namespace seldon

namespace "seldon" created


In [12]:
!helm install ../helm-charts/seldon-core --name seldon-core --namespace seldon \
    --set rbac.enabled=false

NAME:   seldon-core
LAST DEPLOYED: Mon Jun 11 19:58:11 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/Deployment
NAME                    DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-apiserver        1        1        1           0          0s
seldon-cluster-manager  1        1        1           0          0s
redis                   1        1        1           0          0s

==> v1/Service
NAME              TYPE       CLUSTER-IP     EXTERNAL-IP  PORT(S)                        AGE
seldon-apiserver  NodePort   10.109.80.190  <none>       8080:31062/TCP,5000:31612/TCP  0s
redis             ClusterIP  10.110.233.69  <none>       6379/TCP                       0s

==> v1/Pod(related)
NAME                                     READY  STATUS             RESTARTS  AGE
seldon-apiserver-6fc74fc494-5mknt        0/1    ContainerCreating  0         0s
seldon-cluster-manager-86c6cb7b95-4bmzn  0/1    ContainerCreating  0         0s
redis-df886d999-htsnf                    0/1 

Install prometheus and grafana for analytics

In [ ]:
!helm install ../helm-charts/seldon-core-analytics --name seldon-core-analytics \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false \
    --set rbac.enabled=false \
    --namespace seldon

Check all services are running before proceeding.

In [ ]:
!kubectl get pods -n seldon

## Set up REST and gRPC methods

Install gRPC modules for the prediction protos.

In [ ]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

Illustration of both REST and gRPC requests. 

In [1]:
import requests
from requests.auth import HTTPBasicAuth
from proto import prediction_pb2
from proto import prediction_pb2_grpc
import grpc
try:
    from commands import getoutput # python 2
except ImportError:
    from subprocess import getoutput # python 3


NAMESPACE='seldon'
MINIKUBE_IP=getoutput('minikube ip')
MINIKUBE_HTTP_PORT=getoutput("kubectl get svc -n "+NAMESPACE+" -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'")
MINIKUBE_GRPC_PORT=getoutput("kubectl get svc -n "+NAMESPACE+" -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[1].nodePort}'")

def get_token():
    payload = {'grant_type': 'client_credentials'}
    response = requests.post(
                "http://"+MINIKUBE_IP+":"+MINIKUBE_HTTP_PORT+"/oauth/token",
                auth=HTTPBasicAuth('oauth-key', 'oauth-secret'),
                data=payload)
    print(response.text)
    token =  response.json()["access_token"]
    return token

def rest_request():
    token = get_token()
    headers = {'Authorization': 'Bearer '+token}
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://"+MINIKUBE_IP+":"+MINIKUBE_HTTP_PORT+"/api/v0.1/predictions",
                headers=headers,
                json=payload)
    print(response.text)
    
def grpc_request():
    token = get_token()
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = [3,2],
                values = [1.0,1.0,2.0,3.0,4.0,5.0]
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel(MINIKUBE_IP+":"+MINIKUBE_GRPC_PORT)
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('oauth_token', token)]
    response = stub.Predict(request=request,metadata=metadata)
    print(response)


# Integrating with Kubernetes API

## Validation

Using OpenAPI Schema certain basic validation can be done before the custom resource is accepted.

In [ ]:
!kubectl create -f resources/model_invalid1.json -n seldon

## Normal Operation
A simple example is shown below we use a single prepacked model for illustration. The spec contains a set of predictors each of which contains a ***componentSpec*** which is a Kubernetes [PodTemplateSpec](https://kubernetes.io/docs/api-reference/v1.9/#podtemplatespec-v1-core) alongside a ***graph*** which describes how components fit together.

In [ ]:
!pygmentize resources/model.json

## Create Seldon Deployment

Deploy the runtime graph to kubernetes.

In [ ]:
!kubectl apply -f resources/model.json -n seldon

In [ ]:
!kubectl get seldondeployments -n seldon

In [ ]:
!kubectl describe seldondeployments seldon-deployment-example -n seldon

Get the status of the SeldonDeployment. **When ready the replicasAvailable should be 1**.

In [14]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:test-deployment-fx-market-predictor-0 replicas:1 replicasAvailable:1]]]

## Get predictions

#### REST Request

In [6]:
rest_request()

{"access_token":"5d6cc055-2dce-4aff-9356-bb662f5151b0","token_type":"bearer","expires_in":42707,"scope":"read write"}
{
  "meta": {
    "puid": "f43rqjbdmhorbjfdsevuuocknc",
    "tags": {
    },
    "routing": {
      "random-ab-test": 0
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [2, 1],
      "values": [0.05133579311531625, 0.12823373759251927]
    }
  }
}


#### gRPC Request

In [4]:
grpc_request()

{"access_token":"c533fa2b-5da2-4745-ba55-f9e4a353810e","token_type":"bearer","expires_in":43181,"scope":"read write"}
meta {
  puid: "cgagftb7u1so8bu0eaetm77763"
}
data {
  names: "proba"
  tensor {
    shape: 3
    shape: 1
    values: 0.12823373759251927
    values: 0.39731466202150834
    values: 0.8296760813561542
  }
}



## Update deployment with canary

We will change the deployment to add a "canary" deployment. This illustrates:
 - Updating a deployment with no downtime
 - Adding an extra predictor to run alongside th exsting predictor.
 
 You could manage different traffic levels by controlling the number of replicas of each.

In [ ]:
!pygmentize resources/model_with_canary.json

In [ ]:
!kubectl apply -f resources/model_with_canary.json -n seldon

Check the status of the deployments. Note: **Might need to run several times until replicasAvailable is 1 for both predictors**.

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' -n seldon

#### REST Request

In [ ]:
rest_request()

#### gRPC request

In [ ]:
grpc_request()

## Load test

Start a load test which will post REST requests at 10 requests per second.

In [ ]:
!helm install seldon-core-loadtesting --name loadtest  \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --namespace seldon \
    --repo https://storage.googleapis.com/seldon-charts

You should port-foward the grafana dashboard

```bash
kubectl port-forward $(kubectl get pods -n seldon -l app=grafana-prom-server -o jsonpath='{.items[0].metadata.name}') -n seldon 3000:3000
```

You can then iew an analytics dashboard inside the cluster at http://localhost:3000/dashboard/db/prediction-analytics?refresh=5s&orgId=1. Your IP address may be different. get it via minikube ip. Login with:
 - Username : admin
 - password : password (as set when starting seldon-core above)
 
 The dashboard should look like below:
 
 
 <img src="images/dashboard.png" alt="predictor with canary" title="ml graph"/>

# Tear down

In [ ]:
!helm delete loadtest --purge

In [ ]:
!kubectl delete -f resources/model_with_canary.json -n seldon

In [ ]:
!helm delete seldon-core-analytics --purge

In [ ]:
!helm delete seldon-core --purge

In [ ]:
!helm delete seldon-core-crd --purge